# EA experiments

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import logging
import random

from constants import PATH_PURE_COMPONENTS, PATH_MIXTURES, X
from file_io import load_pickle_file
from nnls_fit_with_x_axis_correction import analyze, nnls_fit_with_interpolated_library
from utils import calculate_signal, nnls_fit, rsme
from correction_models import quadratic_correction

from numpy.random import normal

logging.basicConfig(level=logging.INFO)

## Load data

In [3]:
pure_components = load_pickle_file(PATH_PURE_COMPONENTS)
mixtures_data = load_pickle_file(PATH_MIXTURES)

## Experiment

In [4]:
N_POPULATION = 200
N_SURVIVORS = 50
N_LINEAR_COMBINATIONS = 50
N_MUTATIONS = 50
INIT_GUESS = [0, 0, 0]
INIT_DEVIATIONS = [0.01, 0.1, 1]
N_GENERATIONS = 300
RSME_THRESHOLD = 5

In [5]:
correction_model = quadratic_correction
x_original = X
signal = mixtures_data[0]['signal']
true = mixtures_data[0]['contributions']

In [6]:
parameters = []
for _ in range(N_POPULATION):
    candidate = [normal(loc=mean, scale=stdev) for mean, stdev in zip(INIT_GUESS, INIT_DEVIATIONS)]
    parameters.append(candidate)
parameters = np.array(parameters) 

min_rsme = float(np.inf)
best_solution = None

In [7]:
for k in range(N_GENERATIONS):

    rsme_values = []
    for candidate in parameters:
        x_target = correction_model(x_original, candidate)
        
        if min(x_target) < -50:
            continue
        if max(x_target) > 150:
            continue
            
        solution, residual = nnls_fit_with_interpolated_library(x_original, x_target, pure_components, signal)
        rsme_value = rsme(residual)
        rsme_values.append(rsme_value)
        if rsme_value < min_rsme:
            min_rsme = rsme_value
            best_solution = solution
            print(k, min_rsme, solution)
    
    if min_rsme < RSME_THRESHOLD:
        break
        
    sorted_indices = np.argsort(rsme_values)
    best_parameters = parameters[sorted_indices][:N_SURVIVORS]

    best_parameters_mean = np.mean(best_parameters, axis=0)
    best_parameters_stdev = np.std(best_parameters, axis=0)

    parameters = []

    parameters.append(best_parameters[0].tolist())

    for _ in range(N_LINEAR_COMBINATIONS):
        candidate = [random.choice(column) for column in best_parameters.T]
        parameters.append(candidate)

    for _ in range(N_MUTATIONS):
        candidate = [normal(loc=mean, scale=stdev) for mean, stdev in zip(best_parameters_mean, INIT_DEVIATIONS)]
        parameters.append(candidate)

    for _ in range(N_POPULATION - len(parameters)):
        candidate = [normal(loc=mean, scale=stdev) for mean, stdev in zip(INIT_GUESS, INIT_DEVIATIONS)]
        parameters.append(candidate)
    
    parameters = np.array(parameters)

print('True: ', true)

0 782.8948428736031 [ 32.77294555 142.56121319   0.        ]
0 296.0289304590176 [0.00000000e+000 3.33681103e+003 2.31159642e+154]
0 248.4414978778072 [ 25.72070422 344.93288968   0.        ]
0 170.36009776060033 [ 25.33017034 389.21560641  73.37610442]
0 131.84213650064075 [ 26.99683756 402.19457584  39.95356257]
0 117.27520781705599 [ 27.70908092 413.45458978  13.64024539]
0 110.24802733971832 [ 27.55718858 417.8069961    0.        ]
0 75.2597088063432 [ 26.2286778  383.61287449  53.80261961]
0 72.94361542610915 [ 28.29037382 408.91649844   0.        ]
0 44.58571643856745 [ 26.76918111 394.71561587  28.03989046]
0 36.095439858245314 [ 27.86233184 400.28205792   5.39664079]
2 27.282386207324134 [ 27.27546165 394.71386312  19.0713757 ]
3 21.408224029408146 [ 26.81929108 392.73278333   1.76162931]
3 17.496372707235714 [ 26.0549901  387.63114649  26.35290932]
16 12.468555053467085 [ 26.38271895 389.89142473  20.14817108]
17 10.382034274989055 [ 26.55696267 387.93081885  21.9950327 ]
27 1